<a href="https://colab.research.google.com/github/Henrykim1111/tensorflow-developer-certificate/blob/main/TF3_New_Beans_(2022_11_%EC%8B%A0%EA%B7%9C%EC%B6%9C%EC%A0%9C%EB%AC%B8%EC%A0%9C)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
####################################################
## [중요]
## 2023년 6월 1일 기준
## Google Colab에 설치된 텐서플로우(TensorFlow) 버전은 2.12 버전입니다.
## 시험에 제출 가능한 버전은 2.9.0 버전이기 때문에 버전을 다운그레이드 진행해야 합니다.
## 반드시 아래 코드를 실행하여 버전 다운그레이드 후 모델링을 진행하세요.
## 시험 볼 때는 현재 이 코드는 지워주셔야 합니다.
## 관련하여 궁금하신 점은 슬랙 커뮤니티에 질문 남겨 주세요.
####################################################
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

print(f'설치 완료 후 TensorFlow 버전: {tf.__version__}')
print(f'설치 완료 후 TensorFlow Datasets 버전: {tfds.__version__}')

## 시험을 위한 버전 확인 ###########
## TensorFlow:          2.9.0 #
## TensorFlow Datasets: 4.6.0 #
###############################

설치환경: Google Colab
TensorFlow 시험환경을 구성중입니다. 잠시만 기다려 주세요.
(설치는 약 1~5분 정도 소요 됩니다)
[알림] TensorFlow 시험환경 구성이 완료 되었습니다.


/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


설치 완료 후 TensorFlow 버전: 2.9.0
설치 완료 후 TensorFlow Datasets 버전: 4.6.0


In [ ]:
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# COMPUTER VISION WITH CNNs
#
# Create and train a classifier to classify images between three categories
# of beans using the beans dataset.
# ==============================================================================
# ABOUT THE DATASET
#
# Beans dataset has images belonging to 3 classes as follows:
# 2 disease classes (Angular leaf spot, bean rust)
# 1 healthy class (healthy).
# The images are of different sizes and have 3 channels.
# ==============================================================================
#
# INSTRUCTIONS
#
# We have already divided the data for training and validation.
#
# Complete the code in following functions:
# 1. preprocess()
# 2. solution_model()
#
# Your code will fail to be graded if the following criteria are not met:
# 1. The input shape of your model must be (300,300,3), because the testing
#    infrastructure expects inputs according to this specification. You must
#    resize all the images in the dataset to this size while pre-processing
#    the dataset.
# 2. The last layer of your model must be a Dense layer with 3 neurons
#    activated by softmax since this dataset has 3 classes.
#
# HINT: Your neural network must have a validation accuracy of approximately
# 0.75 or above on the normalized validation dataset for top marks.
#


import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# Use this constant wherever necessary
IMG_SIZE = 300

# This function normalizes and resizes the images.

# COMPLETE THE CODE IN THIS FUNCTION
def preprocess(image, label):
    image = tf.image.resize(image, size=(300,300))
    image /= 255
    return image, label


# This function loads the data, normalizes and resizes the images, splits it into
# train and validation sets, defines the model, compiles it and finally
# trains the model. The trained model is returned from this function.

# COMPLETE THE CODE IN THIS FUNCTION.
def solution_model():
    # Loads and splits the data into training and validation splits using tfds.
    (ds_train, ds_validation), ds_info = tfds.load(
        name='beans',
        split=['train', 'validation'],
        as_supervised=True,
        with_info=True)

    BATCH_SIZE = 32

    # Resizes and normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    ds_train = ds_train.map(preprocess).cache().shuffle(
        ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(
        tf.data.experimental.AUTOTUNE)
    ds_validation = ds_validation.map(preprocess).batch(BATCH_SIZE).cache(

    ).prefetch(tf.data.experimental.AUTOTUNE)

    transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
    transfer_model.trainable = False

    # Code to define the model
    model = tf.keras.models.Sequential([
		    transfer_model,
		    Flatten(),
		    Dropout(0.25),
		    Dense(128, activation='relu'),
		    Dropout(0.25),
		    Dense(32, activation='relu'),
		    tf.keras.layers.Dense(3, activation='softmax'),
    ])

    # Code to compile and train the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

    checkpoint_path = "my_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

    model.fit(ds_train,
          validation_data=ds_validation,
          epochs=20,
          callbacks=[checkpoint],
          )

    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/20
20/33 [=================>............] - ETA: 4:21 - loss: 1.8498 - acc: 0.4531